In [1]:
# save in a format readable by c

import numpy as np
import torch

data = torch.load('pytorch_model.bin', map_location='cpu')

In [6]:
data.keys()

odict_keys(['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight', 'h.3.ln_1.bias', 'h.3.attn.bias', 'h.3.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.3.attn.c_pr

In [ ]:
import os
os.makedirs("gpt_weights", exist_ok=True)
for key in data.keys():
    if "attn.c_attn" in key:
        # split to q, k, v
        q, k, v = np.split(data[key].numpy(), 3, axis=-1)
        q.tofile(f"gpt_weights/{key}.q.bin")
        k.tofile(f"gpt_weights/{key}.k.bin")
        v.tofile(f"gpt_weights/{key}.v.bin")
    else:
        data[key].numpy().tofile(f"gpt_weights/{key}.bin")
        # print(data[key].numpy().shape)

# save transpose of wte.weight as etw.weight

etw = data['wte.weight'].numpy().T
etw.tofile("gpt_weights/etw.weight.bin")

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
# import model for text generation
from transformers import AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')

In [50]:
text = "Hello, my name is"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input['input_ids'].shape)
output = model(**encoded_input, output_hidden_states=True)
tokenizer.decode(output.logits.argmax(2)[0][-1])

torch.Size([1, 5])


' John'

In [51]:
# save output['hidden_states'][0][0]

output['hidden_states'][0][0].detach().numpy().tofile("gpt_weights/input.bin")
output.logits[0].detach().numpy().tofile("gpt_weights/output.bin")